### 打码方法
手机号

In [ ]:
def remove_spaces(string: str):
    # 使用 replace() 函数将所有空格替换为空字符串
    try:
        result = string.replace(" ", "")
        result = string.replace("-", "")
        result = string.replace("_", "")
        return result
    except Exception as e:
        print("Remove Space Error: %s" % e)
        return string
        

def mask_numbers_in_text(string: str):
    string_nospace = remove_spaces(string)
    pos_list = [i for i in range(len(string_nospace)) if str.isdigit(string_nospace[i])]
    if len(pos_list) == 0:
        return string
    pos_info = []
    count = 1
    start = pos_list[0]
    # 寻找连续数字
    for i in range(1, len(pos_list)):
        if pos_list[i] - pos_list[i-1] == 1:
            count += 1
            continue
        else:
            end = pos_list[i-1]
            if start != end:
                resp = {"start": start, "len": count}
                pos_info.append(resp)
                count = 1
            start = pos_list[i]
    end = pos_list[-1]
    if start != end:
        resp = {"start": start, "len": count}
        pos_info.append(resp)
    # print(pos_info)
    # 抹掉长数字
    for item in pos_info:
        if item["len"] < 7:
            # print(f"skipped: {item}")
            continue
        mask = "*" * item["len"]
        start = item["start"]
        end = item["start"] + item["len"]
        # print(start, end)
        string_nospace = string_nospace[:start] + mask + string_nospace[end:]
    return string_nospace


q = "027-87680961周一至周日10:00-18:00湖北武汉市湖北省武汉市珞瑜路10号街道口银泰创意城4楼4F006"
r = mask_numbers_in_text(q)
print(r)


邮箱

In [ ]:
def is_valid_email(email):
    if "@" in email and "." in email.split("@")[1]:
        at_index = email.index("@")
        dot_index = email.index(".", at_index)
        if at_index < dot_index and at_index != 0:
            return True
    return False

def mask_email(email):
    at_index = email.index("@")
    local_part = email[:at_index]
    domain_part = email[at_index+1:]
    
    # Mask the local part except the first and last character
    masked_local_part = local_part[0] + "****"
    
    # Mask the domain part except the first character and the top-level domain
    domain_name, tld = domain_part.rsplit(".", 1)
    masked_domain_part = "****" + "." + tld
    
    masked_email = masked_local_part + "@" + masked_domain_part
    return masked_email

def mask_emails_in_text(text):
    words = text.split()
    masked_words = []
    for word in words:
        if is_valid_email(word):
            masked_words.append(mask_email(word))
        else:
            masked_words.append(word)
    return " ".join(masked_words)


q = "你好今天天气咋样"
r = mask_emails_in_text(q)
print(r)


ip地址

In [ ]:
import socket

def is_valid_ip(ip):
    try:
        socket.inet_aton(ip)
        return True
    except socket.error:
        return False

def mask_ip_in_text(text):
    masked_text = ""
    i = 0
    while i < len(text):
        found_ip = False
        # 尝试从当前位置检查长度为7到15的子字符串（符合IP地址格式的最短和最长长度）
        for length in range(15, 6, -1):
            if i + length <= len(text):
                potential_ip = text[i:i + length]
                if is_valid_ip(potential_ip):
                    masked_text += '***.***.***.***'
                    i += length
                    found_ip = True
                    break
        if not found_ip:
            masked_text += text[i]
            i += 1
    return masked_text

'''
input_text = "我的电脑ip地址是192.168.1.1还有一个是10.0.0.1."
output_text = mask_ip_in_text(input_text)
print(output_text)
'''

随机编码（不好用，会码多）

In [ ]:
def is_valid_code_char(c):
    return c.isdigit() or c.isalpha()

def is_random_code(substring):
    if 6 <= len(substring) <= 15:
        return all(is_valid_code_char(c) for c in substring)
    return False

def mask_random_codes(text):
    masked_text = ""
    i = 0
    while i < len(text):
        found_code = False
        # 检查长度为6到15的子字符串
        for length in range(15, 5, -1):
            if i + length <= len(text):
                potential_code = text[i:i + length]
                if is_random_code(potential_code):
                    masked_text += '*' * length
                    i += length
                    found_code = True
                    break
        # 如果不是随机编码，保留当前字符
        if not found_code:
            masked_text += text[i]
            i += 1
    return masked_text

'''
input_text = "这个字符串中有随机编码98QYHQF98QHJ和hkjhy8310o以及一些其他文本。"
output_text = mask_random_codes(input_text)
print(output_text)
'''

地址

In [6]:
import cpca


def mask_addr_in_text(string: str):
    str_list = [string]
    loc_df = cpca.transform(str_list)

    for col in loc_df.columns:
        value = loc_df.at[0, col]
        if str(value) in string:
            if col == '省':
                string = string.replace(value, '**省')
            elif col == '市':
                string = string.replace(value, '**市')
            elif col == '区':
                string = string.replace(value, '**区')
            else:
                string = string.replace(value, '*****')
    return string

q = "027-87680961周一至周日10:00-18:00湖北武汉市湖北省武汉市珞瑜路10号街道口银泰创意城4楼4F006"
r = mask_addr_in_text(q)
print(r)

027-87680961周一至周日10:00-18:00湖北**市**省**市*****


姓名

### 运行数据集测试效果

In [ ]:
import pandas as pd
import os

DATA_DIR = "data"
OUTPUT_DIR = "data_masked"
FILE_NAME  = "文本交付内容2023-12-15至2023-12-21-10329.xlsx"

excel_files = os.listdir(DATA_DIR)
csv_files = os.listdir(OUTPUT_DIR)
'''
for f in excel_files:
    if '$' in f:
        continue
        '''
f = "文本交付内容2023-12-15至2023-12-21-10329.xlsx"
df = pd.read_excel(os.path.join(DATA_DIR, f))
masked_msgs = []
for index, row in df.iterrows():
    print(index)
    if row['谁说'] != "客户":
        continue
    msg = row['消息正文']
    print(msg)
    if not isinstance(msg, str):
        msg = str(msg)
        print("msg not str")
    # 手机号邮箱打码处理
    masked = mask_numbers_in_text(msg)
    masked = mask_emails_in_text(masked)
    masked = mask_addr_in_text(masked)
    if "****" in masked and "****" not in msg:
        df.at[index, '消息正文'] = masked
        print("masked: %s" % masked)

df.to_csv(os.path.join(OUTPUT_DIR, f.split(".xlsx")[0] + ".csv"), index=False, encoding="utf_8_sig")
print(f"############### 完成处理：{f} ###############")
    
print("DONE")